<a href="https://colab.research.google.com/github/RaedHadad/Elephant/blob/main/Elephant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install reportlab
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 5.6 MB/s eta 0:00:00


In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import requests
import json
import datetime
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors
import base64
import matplotlib.pyplot as plt

# Define global variables to hold state
uploaded_data = None
filtered_df = pd.DataFrame()  # Initialize filtered_df as an empty DataFrame

# Tailwind CSS to style and center elements
tailwind_css = """
<link href="https://cdnjs.cloudflare.com/ajax/libs/tailwindcss/2.2.19/tailwind.min.css" rel="stylesheet">
<style>
    body {
        background-color: #f4f4f9;
        font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;
        color: #333;
    }
    .centered {
        display: flex;
        justify-content: center;
        align-items: center;
        height: 100vh;
        flex-direction: column;
        text-align: center;
    }
    .form-container {
        background-color: #ffffff;
        padding: 20px;
        border-radius: 8px;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        max-width: 350px;
        width: 100%;
        margin: auto;
    }
    .page-title {
        color: #1e3a8a; /* Dark blue color */
        font-size: 2rem;
        font-weight: bold;
        margin-bottom: 1rem;
    }
    .form-title {
        color: #1e3a8a; /* Dark blue color */
        font-size: 1.5rem;
        font-weight: bold;
        margin-bottom: 1rem;
    }
    .form-label {
        color: #333;
        font-weight: bold;
        margin-bottom: 0.5rem;
        display: block;
    }
    .form-input {
        width: 100%;
        padding: 10px;
        margin-bottom: 1rem;
        border: 1px solid #ccc;
        border-radius: 4px;
        box-sizing: border-box;
    }
    .form-button {
        width: 100%;
        padding: 10px;
        background-color: #1e7e34; /* More vibrant green color */
        color: white;
        border: none;
        border-radius: 4px;
        cursor: pointer;
        font-weight: bold;
        text-transform: uppercase;
    }
    .form-button:hover {
        background-color: #218838; /* Darker green color */
    }
</style>
"""

# Inject Tailwind CSS
display(HTML(tailwind_css))

# Define the login page elements
username = widgets.Text(
    value='',
    placeholder='Enter your username',
    layout=widgets.Layout(width='30%', padding='5px', margin='0 0 1rem 0', border='1px solid #ccc', border_radius='2px')
)
password = widgets.Password(
    value='',
    placeholder='Enter your password',
    layout=widgets.Layout(width='30%', padding='5px', margin='0 0 1rem 0', border='1px solid #ccc', border_radius='2px')
)
login_button = widgets.Button(
    description='Login',
    disabled=False,
    button_style='',
    tooltip='Click to login',
    icon='check',
    layout=widgets.Layout(width='20%', padding='5px', margin='5px 0'),
    style=dict(button_color='#21ab0f', font_weight='bold', text_transform='uppercase')
)
login_message = widgets.Label()

# Function to check login credentials
def on_login_button_clicked(b):
    if username.value == 'admin' and password.value == 'admin':
        clear_output()
        display_upload_page()
    else:
        login_message.value = 'Invalid username or password. Please try again.'

login_button.on_click(on_login_button_clicked)

# Display the login page elements
def display_login_page():
    clear_output()
    login_message.value = ''
    display(widgets.VBox([
        widgets.HTML("<h1 class='page-title' style='color:#1e3a8a;'>Elephant Application</h1>"),
        widgets.HTML("<h2 class='form-title'>Login</h2>"),
        widgets.HTML("<div class='form-container'>"),
        widgets.HTML("<label class='form-label'>Username</label>"),
        username,
        widgets.HTML("<label class='form-label'>Password</label>"),
        password,
        login_button,
        login_message,
        widgets.HTML("</div>")
    ], layout=widgets.Layout(align_items='center', justify_content='center', height='100vh')))

display_login_page()

# Define the upload page elements
upload_button = widgets.FileUpload(
    accept='.json',
    multiple=False
)
upload_button_description = widgets.Label(value='Upload a JSON file to replace the old one in the database.')
proceed_button = widgets.Button(
    description='Proceed to Search Page',
    disabled=False,
    button_style='success',
    tooltip='Click to proceed to the search page',
    icon='arrow-right'
)
back_button = widgets.Button(
    description='Back to Login',
    disabled=False,
    button_style='warning',
    tooltip='Click to go back to login page',
    icon='arrow-left'
)
upload_message = widgets.Label()

# Function to handle file upload
def on_upload_button_change(change):
    global uploaded_data
    for filename, file_info in upload_button.value.items():
        uploaded_data = json.loads(file_info['content'].decode('utf-8'))
        # Here, you would upload the data to your Firebase Realtime Database
        url = 'https://elephant-994c6-default-rtdb.firebaseio.com/.json'
        requests.put(url, json=uploaded_data)
        upload_message.value = 'File uploaded successfully.'

upload_button.observe(on_upload_button_change, names='value')

# Function to proceed to search page
def on_proceed_button_clicked(b):
    if uploaded_data is not None:
        clear_output()
        display_search_page()
    else:
        upload_message.value = 'Please upload a JSON file before proceeding.'

proceed_button.on_click(on_proceed_button_clicked)

# Function to display the upload page
def display_upload_page():
    clear_output()
    display(widgets.VBox([
        widgets.HTML("<h1 style='color:#1e3a8a; font-size:2rem; font-weight:bold; margin-bottom:1rem;'>Upload Page</h1>"),
        upload_button_description,
        upload_button,
        upload_message,
        widgets.HBox([proceed_button, back_button], layout=widgets.Layout(justify_content='center', gap='10px'))
    ], layout=widgets.Layout(align_items='center', justify_content='center', height='100vh')))

# Function to go back to login page
def on_back_button_clicked(b):
    display_login_page()

back_button.on_click(on_back_button_clicked)

# Define the search page elements
back_to_upload_button = widgets.Button(
    description='Back to Upload',
    disabled=False,
    button_style='warning',
    tooltip='Click to go back to upload page',
    icon='arrow-left'
)

# Function to go back to the upload page from the search page
def on_back_to_upload_button_clicked(b):
    display_upload_page()

back_to_upload_button.on_click(on_back_to_upload_button_clicked)

# Function to generate PDF from DataFrame
def generate_pdf(dataframe, filename='filtered_data.pdf'):
    doc = SimpleDocTemplate(filename, pagesize=letter)
    elements = []
    data = [dataframe.columns.tolist()] + dataframe.values.tolist()
    table = Table(data)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black),
    ]))
    elements.append(table)
    doc.build(elements)

# Function to download PDF
def download_pdf_button_clicked(b):
    if not filtered_df.empty:
        generate_pdf(filtered_df)
        with open('filtered_data.pdf', 'rb') as f:
            pdf_data = f.read()
        b64_pdf = base64.b64encode(pdf_data).decode()
        display(HTML(f'<a href="data:application/pdf;base64,{b64_pdf}" download="filtered_data.pdf">Download PDF</a>'))

download_pdf_button = widgets.Button(
    description='Download as PDF',
    disabled=False,
    button_style='success',
    tooltip='Click to download filtered results as PDF',
    icon='download'
)
download_pdf_button.on_click(download_pdf_button_clicked)

# Function to show graphs
def show_graphs_button_clicked(b):
    if not filtered_df.empty:
        clear_output(wait=True)
        display_search_page_elements()
        display_table(filtered_df)
        display_graphs(filtered_df)

show_graphs_button = widgets.Button(
    description='Show Graphs',
    disabled=False,
    button_style='info',
    tooltip='Click to show graphs for filtered results',
    icon='bar-chart'
)
show_graphs_button.on_click(show_graphs_button_clicked)

# Function to display graphs
def display_graphs(dataframe):
    plt.figure(figsize=(10, 6))

    # Example graph: Number of entries per User
    user_counts = dataframe['User'].value_counts()
    user_counts.plot(kind='bar')
    plt.title('Number of Entries per User')
    plt.xlabel('User')
    plt.ylabel('Number of Entries')
    plt.xticks(rotation=45)
    plt.show()

    plt.figure(figsize=(10, 6))

    # Example graph: Number of entries per Tab
    tab_counts = dataframe['Tab'].value_counts()
    tab_counts.plot(kind='bar', color='orange')
    plt.title('Number of Entries per Tab')
    plt.xlabel('Tab')
    plt.ylabel('Number of Entries')
    plt.xticks(rotation=45)
    plt.show()

    plt.figure(figsize=(10, 6))

    # Example graph: Number of entries per Document
    doc_counts = dataframe['Document'].value_counts()
    doc_counts.plot(kind='bar', color='green')
    plt.title('Number of Entries per Document')
    plt.xlabel('Document')
    plt.ylabel('Number of Entries')
    plt.xticks(rotation=45)
    plt.show()

    plt.figure(figsize=(10, 6))

    # Example graph: Entries over Time
    dataframe['Time'].dt.to_period('M').value_counts().sort_index().plot(kind='line', marker='o')
    plt.title('Entries Over Time')
    plt.xlabel('Time')
    plt.ylabel('Number of Entries')
    plt.xticks(rotation=45)
    plt.show()

    plt.figure(figsize=(10, 6))

    # Example pie chart: Distribution of entries per User
    user_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90)
    plt.title('Distribution of Entries per User')
    plt.ylabel('')
    plt.show()

    plt.figure(figsize=(10, 6))

    # Example pie chart: Distribution of entries per Tab
    tab_counts.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'])
    plt.title('Distribution of Entries per Tab')
    plt.ylabel('')
    plt.show()

# Function to display buttons
def display_buttons():
    display(widgets.HBox([back_to_upload_button, download_pdf_button, show_graphs_button]))

# Define the search page
def display_search_page():
    global from_date, to_date, tab_selector, user_selector, description_input
    # Use the uploaded_data here if available
    global uploaded_data, filtered_df
    if uploaded_data is None:
        # If no data is uploaded yet, fetch the existing data
        url = 'https://elephant-994c6-default-rtdb.firebaseio.com/.json'
        response = requests.get(url)
        data = response.json()
    else:
        data = uploaded_data

    # Convert JSON data to DataFrame
    df = pd.DataFrame(data)

    # Convert the 'Time' column to datetime
    df['Time'] = pd.to_datetime(df['Time'])

    # Get unique values for dropdowns
    unique_tabs = df['Tab'].unique().tolist()
    unique_tabs.insert(0, "All")  # Add an option to select all tabs

    unique_users = df['User'].unique().tolist()
    unique_users.insert(0, "All")  # Add an option to select all users

    # Create date picker widgets
    from_date = widgets.DatePicker(
        description='From',
        disabled=False
    )
    to_date = widgets.DatePicker(
        description='To',
        disabled=False
    )

    # Create combo boxes for Tabs and Users, and a text input for Description
    tab_selector = widgets.Dropdown(
        options=unique_tabs,
        value="All",
        description='Tab:',
        disabled=False
    )

    user_selector = widgets.Dropdown(
        options=unique_users,
        value="All",
        description='User:',
        disabled=False
    )

    description_input = widgets.Text(
        value='',
        placeholder='Enter description to search',
        description='Description:',
        disabled=False
    )

    # Display widgets and back button
    display_search_page_elements()

    # Function to filter data and display table
    def filter_data(change):
        global filtered_df  # Ensure this modifies the global variable
        if from_date.value and to_date.value:
            start_date = pd.to_datetime(from_date.value)
            end_date = pd.to_datetime(to_date.value)
            selected_tab = tab_selector.value
            selected_user = user_selector.value
            description_text = description_input.value.lower()

            filtered_df = df[(df['Time'] >= start_date) & (df['Time'] <= end_date)]
            if selected_tab != "All":
                filtered_df = filtered_df[filtered_df['Tab'] == selected_tab]
            if selected_user != "All":
                filtered_df = filtered_df[filtered_df['User'] == selected_user]
            if description_text:
                filtered_df = filtered_df[filtered_df['Description'].str.lower().str.contains(description_text)]

            # Display the filtered DataFrame
            clear_output(wait=True)
            display_search_page_elements()
            display_table(filtered_df)

    # Add event listeners to date picker, combo box, and text input widgets
    from_date.observe(filter_data, names='value')
    to_date.observe(filter_data, names='value')
    tab_selector.observe(filter_data, names='value')
    user_selector.observe(filter_data, names='value')
    description_input.observe(filter_data, names='value')

# Function to display search page elements
def display_search_page_elements():
    display(widgets.VBox([
        widgets.HTML("<h1 style='color:#1e3a8a; font-size:2rem; font-weight:bold; margin-bottom:1rem;'>Search Page</h1>"),
        from_date,
        to_date,
        tab_selector,
        user_selector,
        description_input,
        widgets.HBox([back_to_upload_button, download_pdf_button, show_graphs_button], layout=widgets.Layout(justify_content='center', gap='10px'))
    ], layout=widgets.Layout(align_items='center', justify_content='center', height='100vh')))

# Function to display the table
def display_table(dataframe):
    display(widgets.HTML("<div class='table-section'>"))
    display_paginated(dataframe)
    display(widgets.HTML("</div>"))

# Function to display paginated data
def display_paginated(dataframe, rows_per_page=10):
    import ipywidgets as widgets
    from IPython.display import display, clear_output

    def display_page(page):
        clear_output(wait=True)
        display_search_page_elements()
        start = page * rows_per_page
        end = start + rows_per_page
        table_html = dataframe.iloc[start:end].to_html(classes='table table-striped table-bordered', index=False)
        display(HTML(f'<style>.dataframe td {{ white-space: nowrap; overflow: hidden; text-overflow: ellipsis; max-width: 400px; }}</style>{table_html}'))
        display(page_slider)

    n_pages = len(dataframe) // rows_per_page + 1
    page_slider = widgets.IntSlider(min=0, max=n_pages - 1, step=1, description='Page')

    def on_page_change(change):
        display_page(change['new'])

    page_slider.observe(on_page_change, names='value')
    display(page_slider)
    display_page(0)
